In [130]:
import time
import os

import numpy as np
import cv2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import pyautogui

In [ ]:
instagram_id = os.getenv('INSTAGRAM_ID')
instagram_password = os.getenv('INSTAGRAM_PASSWORD')
driver_path = os.getenv('DRIVER_PATH')

In [131]:
def subprocess_for_login():
    cancel_button_english()
    cancel_button_korean()
    app_download_alert()
    notification_setting()

def cancel_button_english():
    try:
        button_for_login = driver.find_element_by_xpath("//button[text()='Cancel']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass
    
def cancel_button_korean():
    try:
        button_for_login = driver.find_element_by_xpath("//button[text()='취소']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass

def app_download_alert():
    try:
        button_for_login = driver.find_element_by_xpath("//a[text()='나중에 하기']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass
    
def notification_setting():
    try:
        button_for_login = driver.find_element_by_xpath("//button[text()='나중에 하기']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass

In [132]:
def click_and_write_current_screen(img_background, gry_background, template, sentence, result_file_path = 'instagram/res.png'):
    location = template_match(gry_background, template)
    draw_rectangle(img_background, template, location)
    click_location = get_click_location(template, location)
    cv2.imwrite(result_file_path, img_background)
    pyautogui.click(click_location)
    pyautogui.typewrite(sentence)
    
def click_current_screen(img_background, gry_background, template, result_file_path = 'instagram/res.png'):
    location = template_match(gry_background, template)
    draw_rectangle(img_background, template, location)
    click_location = get_click_location(template, location)
    cv2.imwrite(result_file_path, img_background)
    pyautogui.click(click_location)
    
def template_match(img_grey, template):
    res = cv2.matchTemplate(img_grey, template, cv2.TM_CCOEFF_NORMED)
    threshold = get_threshold(res)
    result = np.where(res > threshold)
    result = zip(*result[::-1])
    return list(result)[0]

def get_click_location(template, res):
    w, h = template.shape[::-1]
    return (res[0] + w / 2, res[1] + h / 2)

def draw_rectangle(original, template, res):
    w, h = template.shape[::-1]
    cv2.rectangle(original, res, (res[0] + w, res[1] + h), (0,0,255), 2)

def get_threshold(nd_array):
    threshold = 0.6
    while not select_specific_position(nd_array, threshold):
        print(threshold)
        threshold += 0.02
    
    return threshold

def select_specific_position(nd_array, threshold):
    tmp_array = np.where(nd_array > threshold)
    if len(tmp_array) == 2:
        return True
    else:
        return False

###  로그인 페이지

target_website = "https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher"

In [168]:
target_website = "https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher"

### 로그인

In [146]:
options = Options()
# chrome user agent를 지정해주기
options.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1")

driver = webdriver.Chrome(driver_path, options=options)

driver.get(target_website)

cookies = driver.get_cookies()

driver.maximize_window()

input_id = driver.find_element_by_xpath("//input[@aria-label='전화번호, 사용자 이름 또는 이메일'][@name='username'][@type='text']")
input_password = driver.find_element_by_xpath("//input[@aria-label='비밀번호'][@name='password'][@type='password']")


input_id.clear()
input_id.send_keys("amz_ojh@naver.com")
input_password.clear()
input_password.send_keys("wlghksdl78!")
button_for_login = driver.find_element_by_xpath("//button[@type='submit'][text()='로그인']")

button_for_login.click()

### 팝업 핸들링

In [151]:
subprocess_for_login()

### 포스팅을 위한 button object 및 click

In [152]:
button_for_posting = driver.find_element_by_xpath("//div[@role='menuitem'][@tabindex='0']")
button_for_posting.click()

In [154]:
pyautogui.screenshot('instagram/current.png')
img_file_dialog = cv2.imread('instagram/current.png')
grey_file_dialog = cv2.cvtColor(img_file_dialog, cv2.COLOR_BGR2GRAY)
template_for_filedialog = cv2.imread('instagram/file_dialog.png', 0)
template_for_button = cv2.imread('instagram/open.png', 0)

### file path입력

In [155]:
file_path = r"C:\Users\amz_o\Camino.jpg"

In [156]:
click_and_write_current_screen(img_file_dialog, grey_file_dialog, template_for_filedialog, file_path)
click_current_screen(img_file_dialog, grey_file_dialog, template_for_button)

In [158]:
button_for_click = driver.find_element_by_xpath("//button[text()='다음']")

In [159]:
button_for_click.click()

In [164]:
text_area = driver.find_element_by_xpath("//textarea[@aria-label='문구 입력...'][@placeholder='문구 입력...']")

In [165]:
text_area.send_keys("gahawegwefwe")

In [166]:
button_for_post = driver.find_element_by_xpath("//button[text()='공유하기']")
button_for_post.click()